In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from dlomix.constants import ALPHABET_UNMOD
from prosit_t.layers import (
    MetaEncoder,
    FusionLayer,
    RegressorV2,
    PositionalEmbedding,
    TransformerEncoder,
    CrossAttention,
)

2023-09-11 16:38:03.805049: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 16:38:04.544293: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
%load_ext autoreload

%autoreload 2

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [5]:
SEQ_LENGTH = 30
embeddings_count = len(ALPHABET_UNMOD) + 2
max_ion = SEQ_LENGTH - 1
len_fion = 6

In [6]:
embedding_output_dim = 64
num_heads = 16
ff_dim = 32
transformer_dropout = 0.1
num_transformers = 4
dropout_rate = 0
dense_dim_factor = 4

In [7]:
in_sequence = tf.keras.layers.Input(shape=(30,))
in_collision_energy = tf.keras.layers.Input(shape=(6,))
in_precursor_charge = tf.keras.layers.Input(shape=(1,))

encoded_meta = MetaEncoder(embedding_output_dim*dense_dim_factor, dropout_rate)([in_collision_energy, in_precursor_charge])
encoded_meta = tf.keras.layers.Reshape([1, embedding_output_dim*dense_dim_factor])(encoded_meta)
x = preprocessing.StringLookup(vocabulary=list(ALPHABET_UNMOD.keys()))(in_sequence)
x = PositionalEmbedding(
            embeddings_count, embedding_output_dim
        )(x)
x = TransformerEncoder(
            embedding_output_dim, num_heads, ff_dim, rate=transformer_dropout, num_transformers=num_transformers
        )(x)
x = CrossAttention(
            num_heads=num_heads,
            key_dim=embedding_output_dim * dense_dim_factor,
            dropout=dropout_rate)(x=encoded_meta, context=x) #x is query, econded meta - key,value
x = tf.keras.layers.Flatten()(x)
x = RegressorV2(len_fion * max_ion)(x)

model = tf.keras.Model(inputs=[in_sequence, in_collision_energy, in_precursor_charge],outputs=x)
model.summary()

2023-09-11 16:38:06.623745: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38136 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:21:00.0, compute capability: 8.6


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 30)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 6)]                  0         []                            
                                                                                                  
 input_3 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 string_lookup (StringLooku  (None, 30)                   0         ['input_1[0][0]']             
 p)                                                                                           

In [35]:
in_sequence = tf.keras.layers.Input(shape=(30,))
in_collision_energy = tf.keras.layers.Input(shape=(6,))
in_precursor_charge = tf.keras.layers.Input(shape=(1,))

encoded_meta = MetaEncoder(embedding_output_dim*dense_dim_factor, dropout_rate)([in_collision_energy, in_precursor_charge])
# encoded_meta = tf.keras.layers.Reshape([1, embedding_output_dim*dense_dim_factor])(encoded_meta)
x = preprocessing.StringLookup(vocabulary=list(ALPHABET_UNMOD.keys()))(in_sequence)
x = PositionalEmbedding(
            embeddings_count, embedding_output_dim
        )(x)
x = TransformerEncoder(
            embedding_output_dim, num_heads, ff_dim, rate=transformer_dropout, num_transformers=num_transformers
        )(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Concatenate()([x, encoded_meta])
x = tf.keras.layers.Dense(512)(x)
x = RegressorV2(len_fion * max_ion)(x)

model = tf.keras.Model(inputs=[in_sequence, in_collision_energy, in_precursor_charge],outputs=x)
model.summary()

Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_70 (InputLayer)       [(None, 30)]                 0         []                            
                                                                                                  
 string_lookup_22 (StringLo  (None, 30)                   0         ['input_70[0][0]']            
 okup)                                                                                            
                                                                                                  
 positional_embedding_22 (P  (None, 30, 64)               1408      ['string_lookup_22[0][0]']    
 ositionalEmbedding)                                                                              
                                                                                           

In [12]:
in_sequence = tf.keras.layers.Input(shape=(30,))
in_collision_energy = tf.keras.layers.Input(shape=(6,))
in_precursor_charge = tf.keras.layers.Input(shape=(1,))

encoded_meta = MetaEncoder(embedding_output_dim*dense_dim_factor, dropout_rate)([in_collision_energy, in_precursor_charge])
x = preprocessing.StringLookup(vocabulary=list(ALPHABET_UNMOD.keys()))(in_sequence)
x = PositionalEmbedding(
            embeddings_count, embedding_output_dim
        )(x)
x = TransformerEncoder(
            embedding_output_dim, num_heads, ff_dim, rate=transformer_dropout, num_transformers=num_transformers
        )(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(embedding_output_dim * dense_dim_factor)(x)
x = tf.keras.layers.Attention()([x,encoded_meta]) #x is query, econded meta - key,value
x = tf.keras.layers.Flatten()(x)
x = RegressorV2(len_fion * max_ion)(x)

model = tf.keras.Model(inputs=[in_sequence, in_collision_energy, in_precursor_charge],outputs=x)
model.summary()

(None, 256)
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_16 (InputLayer)       [(None, 30)]                 0         []                            
                                                                                                  
 string_lookup_5 (StringLoo  (None, 30)                   0         ['input_16[0][0]']            
 kup)                                                                                             
                                                                                                  
 positional_embedding_5 (Po  (None, 30, 64)               1408      ['string_lookup_5[0][0]']     
 sitionalEmbedding)                                                                               
                                                                                

In [ ]:
in_sequence = tf.keras.layers.Input(shape=(30,))
in_collision_energy = tf.keras.layers.Input(shape=(6,))
in_precursor_charge = tf.keras.layers.Input(shape=(1,))

encoded_meta = MetaEncoder(embedding_output_dim*dense_dim_factor, dropout_rate)([in_collision_energy, in_precursor_charge])

x = preprocessing.StringLookup(vocabulary=list(ALPHABET_UNMOD.keys()))(in_sequence)
x = PositionalEmbedding(
            embeddings_count, embedding_output_dim
        )(x)
x = TransformerEncoder(
            embedding_output_dim, num_heads, ff_dim, rate=transformer_dropout, num_transformers=num_transformers
        )(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(embedding_output_dim*dense_dim_factor)(x)
x = tf.keras.layers.Multiply()([x, encoded_meta])
x = tf.keras.layers.Flatten()(x)
x = RegressorV2(len_fion * max_ion)(x)

model = tf.keras.Model(inputs=[in_sequence, in_collision_energy, in_precursor_charge],outputs=x)
model.summary()